In [127]:
import pandas as pd
import numpy as np
import os
import shutil
from collections import OrderedDict
from datetime import datetime, timedelta
import time
import math
import random

from joblib import dump, load
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix

from tabulate import tabulate

# Get All Input Files

In [72]:
input_path = './predict_inputs/'
input_files = [input_path + file for file in os.listdir(input_path) if file.endswith('DRF')]
print(input_files)

['./predict_inputs/CDX0504.DRF']


# Add Number of Entrants to Input File

In [191]:
def add_entrants(input_file):
    '''
        Find highest post position for each race and use as number of entrants
        
        Args:
            file (string): path to results file
            
        Returns:
            Nothing
    '''
    # Load files 
    input_df = pd.read_csv(input_file, header=None)
    
    # Find how many races are in each
    race_col = 2
    num_input_races = input_df[race_col].max()
    
    # For each race, count entrants, append as last column
    input_last_col = input_df.columns.max() + 1
    race_entrants = {}
    for race in range(1,num_input_races+1):
        entrants = input_df.loc[input_df[race_col] == race].shape[0]
        iloc = input_df.loc[input_df[race_col] == race].index
        input_df.loc[iloc, input_last_col] = pd.Series(entrants, index=iloc)
        race_entrants[race] = entrants
        
    # Save back to file
    input_df.to_csv(input_file, header=False, index=False)
    
for file in input_files:
    add_entrants(file)

# Parse Files for Input Features

In [193]:
input_map = OrderedDict({
        1: 'date',
        2: 'race_num',
        3: 'post_pos',
        1435: 'num_entrants',
        44: 'horse_name',
        42: 'entry',
        855: 'last_speed_rating',
        216: 'speed_par',
        33: 'app_weight_alw',
        64: 'starts_at_dist',
        96: 'lt_starts',
        100: 'lt_earnings',
        43: 'ml_odds',
        28: 'trainer_starts',
        29: 'trainer_wins',
        34: 'jockey_starts',
        35: 'jockey_wins'
    })

# Iterate through all input files
master_df = pd.DataFrame()
for file in input_files:
    # Open files to dataframe -- Take only columns that are necessary
    input_cols = [k for k in input_map.keys()]
    input_tmp = pd.read_csv(file, header=None)[input_cols]
    
    # Rename cols
    input_tmp.columns = [input_map[col] for col in input_tmp.columns]    
    
    # Get Track Code
    #track_code = file[file.rfind('/')+1:][:3]
    #input_tmp['track_code'] = track_code
    
    # Add these inputs/results to dataframes
    master_df = master_df.append(input_tmp)
    
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,jockey_starts,jockey_wins,last_speed_rating,speed_par,num_entrants,trainer_starts,trainer_wins
0,21,20190504,1,1,626700,NaN,35,1,12.0,RECOUNT,14,1,89.0,99,7.0,0,0
1,21,20190504,1,2,482823,NaN,36,2,2.5,CONTROL STAKE,7,2,88.0,99,7.0,9,2
2,5,20190504,1,3,138700,NaN,5,3,2.5,CURATE,19,2,95.0,99,7.0,3,0
3,6,20190504,1,4,298244,NaN,17,4,6.0,LOOSE ON THE TOWN,10,3,87.0,99,7.0,0,0
4,3,20190504,1,5,183450,NaN,11,5,10.0,KOWBOY KARMA,12,4,83.0,99,7.0,1,0


# Derive Additional Features

In [195]:
def get_jockey_win_pct(df):
    df['jockey_win_pct'] = df['jockey_wins'] / df['jockey_starts']
    return df.drop(['jockey_wins', 'jockey_starts'], axis=1)

def get_trainer_win_pct(df):
    df['trainer_win_pct'] = df['trainer_wins'] / df['trainer_starts']
    return df.drop(['trainer_wins', 'trainer_starts'], axis=1)


# Calculate jockey/trainer win percentages
master_df = get_jockey_win_pct(master_df)
master_df = get_trainer_win_pct(master_df)

# Clean NaN cols
master_df = master_df.fillna(value=0)
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,last_speed_rating,speed_par,num_entrants,jockey_win_pct,trainer_win_pct
0,21,20190504,1,1,626700,0.0,35,1,12.0,RECOUNT,89.0,99,7.0,0.071429,0.000000
1,21,20190504,1,2,482823,0.0,36,2,2.5,CONTROL STAKE,88.0,99,7.0,0.285714,0.222222
2,5,20190504,1,3,138700,0.0,5,3,2.5,CURATE,95.0,99,7.0,0.105263,0.000000
3,6,20190504,1,4,298244,0.0,17,4,6.0,LOOSE ON THE TOWN,87.0,99,7.0,0.300000,0.000000
4,3,20190504,1,5,183450,0.0,11,5,10.0,KOWBOY KARMA,83.0,99,7.0,0.333333,0.000000


# Get Past Performance Data

In [196]:
# Map for past performance fields
input_keys = OrderedDict({
    1: 'date',
    2: 'race_num',
    44: 'horse_name',
})

input_pp_map = OrderedDict({
    615 : 'pp_finish_pos',
    345 : 'pp_num_entrants',
    5: 'todays_distance',
    315: 'pp_distance',
    #535 : 'pp_race_class',
    255 : 'pp_race_date',
    1045: 'pp_claimed',
    1125: 'pp_favorite',
    605: 'pp_stretch_pos',
    113: 'pp_workout_time',
    465: 'pp_winners_margin'
})

def get_past_performance_data(input_files, num_races=3):
    '''
        Go through input files and get past performance data
        
        Args: 
            input_files (list): list of input file names 
            num_races (int): number of races back to grab data for
            
        Returns:
            pd.DataFrame with past performance data for each horse
    '''
    pp_df = pd.DataFrame()
    
    # Create new mapping for this number of races
    updated_pp_map = OrderedDict({k+race:'{}_{}'.format(v, race) for k,v in input_pp_map.items() for race in range(num_races)})
    
    for file in input_files:
        # Open input file to df
        df = pd.read_csv(file, header=None)
        
        # Only include key/past performance columns
        incl_cols = list(input_keys.keys()) + list(updated_pp_map.keys())
        df = df[incl_cols]
        
        # Rename columns
        renamed_cols = []
        for col in df.columns:
            try:
                # PP Column
                rn_col = updated_pp_map[col]
            except:
                # Key Column 
                rn_col = input_keys[col]    
            renamed_cols.append(rn_col)
        df.columns = renamed_cols
        
        # Append this file's df to pp_df
        pp_df = pp_df.append(df)
    
    return pp_df

num_races = 3
pp_df = get_past_performance_data(input_files, num_races)
pp_df.head()

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,pp_finish_pos_1,pp_finish_pos_2,pp_favorite_2,pp_workout_time_0,pp_workout_time_1,pp_workout_time_2,pp_distance_0,pp_distance_1,pp_distance_2,pp_race_date_0
0,20190504,1,RECOUNT,20180428.0,20180329.0,1320,D,NaN,1.75,1.00,...,1,8.0,0.0,49.2,62.6,61.4,1320.0,1320.0,1320.0,20180602.0
1,20190504,1,CONTROL STAKE,20190209.0,20190119.0,1320,D,NaN,1.50,0.50,...,2,2.0,0.0,48.4,48.0,-61.4,1320.0,1320.0,1320.0,20190309.0
2,20190504,1,CURATE,20190228.0,20181110.0,1320,D,NaN,0.06,2.00,...,1,4.0,1.0,49.2,48.6,50.2,1320.0,1320.0,1320.0,20190412.0
3,20190504,1,LOOSE ON THE TOWN,20180818.0,20180407.0,1320,D,NaN,5.25,0.75,...,8,7.0,0.0,-62.2,59.4,60.4,1430.0,1320.0,1210.0,20180915.0
4,20190504,1,KOWBOY KARMA,20181103.0,20180914.0,1320,D,NaN,0.25,2.00,...,6,1.0,0.0,60.8,59.8,48.2,1320.0,1540.0,1540.0,20190119.0


# Clean Past Performance Data

In [197]:
def clean_finish_pos_cols(pp_df, num_races=3):
        # Get finish position column names
        finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
        
        def clean_val(val):
            # If value is numeric, return its integer rep
            try:
                return int(val)
            except:
                # If value not numeric, return 0 (did not finish)
                return int(92)
            
        # Clean columns
        for col in finish_pos_cols:
            pp_df[col] = pp_df[col].apply(clean_val)
            
        return pp_df
    
def clean_stretch_pos_cols(pp_df, num_races=3):
        # Get stretch position column names
        stretch_pos_cols = ['pp_stretch_pos_{}'.format(race) for race in range(num_races)]
        
        def clean_val(val):
            # If value is numeric, return its integer rep
            try:
                return int(val)
            except:
                # If value not numeric, return 92 (did not finish)
                return int(92)
            
        
        # Clean columns
        for col in stretch_pos_cols:
            pp_df[col] = pp_df[col].apply(clean_val)
            
        return pp_df
    
def clean_favorite_cols(pp_df, num_races=3):
    def conv(val):
        if math.isnan(val):
            return int(0)
        else:
            return int(val)
        
    # Get favorite column names
    favorite_columns = ['pp_favorite_{}'.format(race) for race in range(num_races)]
    
    # Convert to integers
    for col in favorite_columns:
        pp_df[col] = pp_df[col].apply(conv)
        
    return pp_df
    
    
def clean_pp_df(pp_df, num_races=3):
    '''
        Clean fields in pp_df
        
        Args:
            pp_df (pd.DataFrame): dataframe to clean
            
        Returns:
            (pd.DataFrame) cleaned version of pp_df
    '''
    pp_df = clean_finish_pos_cols(pp_df, num_races)
    pp_df = clean_stretch_pos_cols(pp_df, num_races)
    pp_df = clean_favorite_cols(pp_df, num_races)
    
    return pp_df

pp_df = clean_pp_df(pp_df, num_races)
pp_df.head(10)

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,pp_finish_pos_1,pp_finish_pos_2,pp_favorite_2,pp_workout_time_0,pp_workout_time_1,pp_workout_time_2,pp_distance_0,pp_distance_1,pp_distance_2,pp_race_date_0
0,20190504,1,RECOUNT,20180428.0,20180329.0,1320,D,NaN,1.75,1.00,...,1,8,0,49.2,62.6,61.4,1320.0,1320.0,1320.0,20180602.0
1,20190504,1,CONTROL STAKE,20190209.0,20190119.0,1320,D,NaN,1.50,0.50,...,2,2,0,48.4,48.0,-61.4,1320.0,1320.0,1320.0,20190309.0
2,20190504,1,CURATE,20190228.0,20181110.0,1320,D,NaN,0.06,2.00,...,1,4,1,49.2,48.6,50.2,1320.0,1320.0,1320.0,20190412.0
3,20190504,1,LOOSE ON THE TOWN,20180818.0,20180407.0,1320,D,NaN,5.25,0.75,...,8,7,0,-62.2,59.4,60.4,1430.0,1320.0,1210.0,20180915.0
4,20190504,1,KOWBOY KARMA,20181103.0,20180914.0,1320,D,NaN,0.25,2.00,...,6,1,0,60.8,59.8,48.2,1320.0,1540.0,1540.0,20190119.0
5,20190504,1,MR. CROW,20180810.0,20180728.0,1320,D,NaN,1.25,2.00,...,5,4,0,51.2,60.6,-47.0,1320.0,1320.0,1320.0,20180929.0
6,20190504,1,ZIPP ON BY,20181229.0,20181122.0,1320,D,NaN,6.75,0.06,...,1,6,0,63.0,51.4,51.2,1320.0,1320.0,1210.0,20190406.0
7,20190504,2,GENERAL DOWNS,20190216.0,20190112.0,1760,D,NaN,1.50,0.25,...,4,1,0,48.8,61.4,-48.8,1980.0,1980.0,-1870.0,20190316.0
8,20190504,2,MY BOY JACK,20180505.0,20180414.0,1760,D,NaN,0.13,2.50,...,5,1,1,-74.6,74.8,62.6,2200.0,2200.0,1870.0,20180707.0
9,20190504,2,WAR GIANT,20180922.0,20180630.0,1760,D,NaN,3.75,0.25,...,8,1,1,61.8,61.4,52.2,1760.0,1320.0,1540.0,20190322.0


# Derive Metrics with Cleaned Data

In [198]:
def get_improved_finish_pos(pp_df, num_races=3):
    '''
        Calculate if horse is improving its finish position over the past races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            
        Returns:
            (pd.DataFrame) an updated version of pp_df with a column for improved
            finish position
    '''
    
    def all_seq_vals_less_than_one(seq):
        ''' Returns true if all values in the passed sequence are <=1 '''
        for elt in seq:
            if elt > 1:
                return False
        return True

    # Get past finish pos cols
    finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
    # Concat all finish positions for each horse in a sequential list starting with most
    # recent race 
    finish_pos_seqs = pp_df[finish_pos_cols].values.tolist()
    
    imp_finish_pos = []
    for seq in finish_pos_seqs:
        # Skip any sequence where NaN appears
        if np.nan in seq:
            imp_finish_pos.append(False)
        # Skip any seq where horse didn't finish (fractional finish pos > 1)
        elif not all_seq_vals_less_than_one(seq):
            imp_finish_pos.append(False)
        else:
            # Determine if last finish was better than previous average finish
            most_recent = seq[0]
            avg_prev_finish = np.mean(seq[1:])
            imp_finish_pos.append(most_recent < avg_prev_finish)
            
    # Assign improved finish position series to pp_df
    pp_df['imp_finish_pos'] = pd.Series(imp_finish_pos)
    
    return pp_df

def get_recent_race(pp_df, threshold=21):
    '''
        Determine if a horse has had a previous race within threshold days
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            threshold (int): days to look back
            
        Returns:
            (pd.DataFrame) updated pp_df containing bool column stating whether
            each horse had a recent race or not
    '''
    
    def conv_date(val):
        # Set NaNs to 1970
        if math.isnan(float(val)):
            val = 19700101
        val = str(int(val))
        return datetime.strptime(val, '%Y%m%d')
    
    # Determine if race most recent race is within threshold
    pp_df['today'] = pp_df['date'].apply(conv_date)
    pp_df['last_race'] = pp_df['pp_race_date_0'].apply(conv_date)
    pp_df['timedelta'] = pd.Series(timedelta(days=threshold), index=pp_df.index)
    pp_df['cutoff'] = pp_df['today'] - pp_df['timedelta']
    pp_df['recent_race'] = pp_df['cutoff'] <= pp_df['last_race'] # Cutoff happened before last race
    
    # Drop intermediate columns
    drop_cols = ['today', 'last_race', 'timedelta', 'cutoff']
    return pp_df.drop(drop_cols, axis=1)
    
def get_past_finish_pos(pp_df, num_races=3):
    '''
        Calculate previous finish positions as finish_pos/num_entrants
        
        Args:
            pp_df (pd.DataFrame) past performance dataframe
            num_races (int): number of races for which we have past performance
        
        Returns:
            (pd.DataFrame) updated pp_df with finish positions calculated
    '''
    # Get finish position/entrants columns names
    finish_pos_cols = ['pp_finish_pos_{}'.format(race) for race in range(num_races)]
    entrants_cols = ['pp_num_entrants_{}'.format(race) for race in range(num_races)]
    cols = list(zip(finish_pos_cols, entrants_cols))
    
    # Divide finish pos by num entrants, save a raw copy of finish position
    for f_col, e_col in cols:
        pp_df['raw_'+f_col] = pp_df[f_col]
        pp_df[f_col] = pp_df[f_col] / pp_df[e_col]
    
    return pp_df

def get_claimed_in_past(pp_df, num_races=3):
    '''
        Determine if a horse was claimed in the past num_races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Claimed column names
    claimed_cols = ['pp_claimed_{}'.format(race) for race in range(num_races)]
    claimed_seqs = pp_df[claimed_cols].values.tolist()
    
    # Determine if claimed in recent races    
    claimed = [True if 'c' in seq else False for seq in claimed_seqs]
    
    # Assign to pp_df
    pp_df['was_claimed'] = pd.Series(claimed, index=pp_df.index).fillna(False)
    
    return pp_df

def was_favorite(pp_df, num_races=3):
    '''
        Determine if a horse was the favorite in any of its recent races
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Get favorite columns
    favorite_cols = ['pp_favorite_{}'.format(race) for race in range(num_races)]
    favorite_seqs = pp_df[favorite_cols].values.tolist()
    
    # If 1 in sequence, horse was favorite in recent race
    was_favorite = [True if 1 in seq else False for seq in favorite_seqs]
    
    pp_df['was_favorite'] = pd.Series(was_favorite, index=pp_df.index)
    
    return pp_df

def improved_stretch_pos(pp_df, num_races=3):
    '''
        Determine if a horse improved its finish position down the stretch in the most
        recent race
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Stretch position columns
    stretch_pos_cols = ['pp_stretch_pos_{}'.format(race) for race in range(num_races)]
    
    # Get fractional stretch position
    pp_df['pp_stretch_pos'] = pp_df['pp_stretch_pos_0'] / pp_df['pp_num_entrants_0']
    
    # Determine if stretch pos > finish pos
    pp_df['improved_stretch_pos'] = pp_df['pp_stretch_pos'] > pp_df['pp_finish_pos_0']
    
    return pp_df.drop(['pp_stretch_pos'], axis=1)

def had_bullet_workout(pp_df, num_races=3):
    '''
        Determine if a horse had a bullet workout in previous workout
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
    '''
    # Get workout time columns
    workout_cols = ['pp_workout_time_{}'.format(race) for race in range(num_races)]
    
    # Determine if last workout was bullet
    pp_df['had_bullet'] = pp_df['pp_workout_time_0'] < 0
    
    return pp_df

def won_by_margin(pp_df, num_races=3, margin=5):
    '''
        Determine if a horse won last race by at least margin lengths
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            margin (float): number of lengths by which horse had to win
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Find all winners of last race
    winners = pp_df.loc[pp_df.raw_pp_finish_pos_0 == 1]
    
    # Find all winners with at least margin length victory
    big_winners = winners.loc[winners.pp_winners_margin_0 >= margin].index
    
    # Create new col, default to false. Set True for all horses in big_winners index
    pp_df['won_by_margin'] = pd.Series(False, index=pp_df.index)
    pp_df.loc[big_winners, 'won_by_margin'] = True
    
    return pp_df

def won_at_similar_distance(pp_df, num_races=3):
    '''
        Determine if a horse won a recent race at the same distance as today's race
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame) updated version of pp_df
    '''
    # Get distance and finish pos cols
    distance_cols = ['pp_distance_{}'.format(race) for race in range(num_races)]
    raw_finish_cols = ['raw_pp_finish_pos_{}'.format(race) for race in range(num_races)]
    
    # Find winners at similar distance by iterating through each recent race
    winners = []
    for race in range(num_races):
        dist_col = 'pp_distance_{}'.format(race)
        finish_col = 'raw_pp_finish_pos_{}'.format(race)
        today_dist_col = 'todays_distance_0'
        
        winners += pp_df.loc[(pp_df[dist_col] == pp_df[today_dist_col]) & (pp_df[finish_col] == 1)].index.tolist()
        
    # Create new column to denote if won at similar distance -- default False, True for 
    # horses in winners
    pp_df['won_at_similar_dist'] = pd.Series(False, index=pp_df.index)
    pp_df.loc[winners, 'won_at_similar_dist'] = True

    return pp_df
    
def derive_pp_metrics(pp_df, num_races=3):
    # Past Finish Positions
    pp_df = get_past_finish_pos(pp_df, num_races)
    # Improved Finish Position
    pp_df = get_improved_finish_pos(pp_df, num_races)
    # Recent race
    pp_df = get_recent_race(pp_df, 21)
    # Claimed in Past
    pp_df = get_claimed_in_past(pp_df, num_races)
    # Favorite in past
    pp_df = was_favorite(pp_df, num_races)
    # Improved stretch pos
    pp_df = improved_stretch_pos(pp_df, num_races)
    # Workout Rating
    pp_df = had_bullet_workout(pp_df, num_races)
    # Won last race by 5+ lengths
    pp_df = won_by_margin(pp_df, num_races, margin=4)
    # Won at similar distance
    pp_df = won_at_similar_distance(pp_df, num_races)    
    
    return pp_df
    
pp_df = derive_pp_metrics(pp_df)
pp_df.head()

,date,race_num,horse_name,pp_race_date_1,pp_race_date_2,todays_distance_0,todays_distance_1,todays_distance_2,pp_winners_margin_0,pp_winners_margin_1,...,raw_pp_finish_pos_1,raw_pp_finish_pos_2,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,20190504,1,RECOUNT,20180428.0,20180329.0,1320,D,NaN,1.75,1.00,...,1,8,False,False,False,True,False,False,False,True
1,20190504,1,CONTROL STAKE,20190209.0,20190119.0,1320,D,NaN,1.50,0.50,...,2,2,False,False,False,False,True,False,False,False
2,20190504,1,CURATE,20190228.0,20181110.0,1320,D,NaN,0.06,2.00,...,1,4,True,False,False,True,False,False,False,True
3,20190504,1,LOOSE ON THE TOWN,20180818.0,20180407.0,1320,D,NaN,5.25,0.75,...,8,7,False,False,False,False,False,True,False,False
4,20190504,1,KOWBOY KARMA,20181103.0,20180914.0,1320,D,NaN,0.25,2.00,...,6,1,False,False,False,False,False,False,False,False


# Clean Fields used in Derivations

In [199]:
def clean_deriv_columns(pp_df, num_races=3):
    '''
        Drop all columns used as intermediates during feature derivation from pp_df
        
        Args:
            pp_df (pd.DataFrame): past performance dataframe
            num_races (int): number of races for which we have past performances
            
        Returns:
            (pd.DataFrame): a cleaned version of pp_df
    '''
    column_dict = {
        'claimed_cols' : ['pp_claimed_{}'.format(race) for race in range(num_races)],
        'date_cols' : ['pp_race_date_{}'.format(race) for race in range(num_races)],
        'distance_cols' : ['pp_distance_{}'.format(race) for race in range(num_races)],
        'entrants_cols' : ['pp_num_entrants_{}'.format(race) for race in range(num_races)],   
        'favorite_cols' : ['pp_favorite_{}'.format(race) for race in range(num_races)],
        'finish_pos_cols' : ['pp_finish_pos_{}'.format(race) for race in range(num_races)],
        'margin_cols' : ['pp_winners_margin_{}'.format(race) for race in range(num_races)],
        'raw_finish_cols' : ['raw_pp_finish_pos_{}'.format(race) for race in range(num_races)],
        'stretch_pos_cols' : ['pp_stretch_pos_{}'.format(race) for race in range(num_races)],
        'todays_distance_cols' : ['todays_distance_{}'.format(race) for race in range(num_races)],
        'workout_cols' : ['pp_workout_time_{}'.format(race) for race in range(num_races)],
    }
    
    for k, cols in column_dict.items():
        pp_df = pp_df.drop(cols, axis=1)
        
    return pp_df

pp_df = clean_deriv_columns(pp_df, num_races)
pp_df.head()

,date,race_num,horse_name,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,20190504,1,RECOUNT,False,False,False,True,False,False,False,True
1,20190504,1,CONTROL STAKE,False,False,False,False,True,False,False,False
2,20190504,1,CURATE,True,False,False,True,False,False,False,True
3,20190504,1,LOOSE ON THE TOWN,False,False,False,False,False,True,False,False
4,20190504,1,KOWBOY KARMA,False,False,False,False,False,False,False,False


# Merge Past Performance with Input Features

In [200]:
master_df = pd.merge(master_df, pp_df,
                     how='left', left_on=['date', 'race_num', 'horse_name'],
                     right_on=['date','race_num', 'horse_name'])
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,jockey_win_pct,trainer_win_pct,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,21,20190504,1,1,626700,0.0,35,1,12.0,RECOUNT,...,0.071429,0.000000,False,False,False,True,False,False,False,True
1,21,20190504,1,2,482823,0.0,36,2,2.5,CONTROL STAKE,...,0.285714,0.222222,False,False,False,False,True,False,False,False
2,5,20190504,1,3,138700,0.0,5,3,2.5,CURATE,...,0.105263,0.000000,True,False,False,True,False,False,False,True
3,6,20190504,1,4,298244,0.0,17,4,6.0,LOOSE ON THE TOWN,...,0.300000,0.000000,False,False,False,False,False,True,False,False
4,3,20190504,1,5,183450,0.0,11,5,10.0,KOWBOY KARMA,...,0.333333,0.000000,False,False,False,False,False,False,False,False


# Convert Boolean Columns to Binary

In [201]:
bool_cols = ['imp_finish_pos', 'recent_race', 'was_claimed','was_favorite', \
             'improved_stretch_pos', 'had_bullet', 'won_by_margin', \
             'won_at_similar_dist']

for col in bool_cols:
    master_df[col] = master_df[col].apply(lambda x: int(x))

master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,jockey_win_pct,trainer_win_pct,imp_finish_pos,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist
0,21,20190504,1,1,626700,0.0,35,1,12.0,RECOUNT,...,0.071429,0.000000,0,0,0,1,0,0,0,1
1,21,20190504,1,2,482823,0.0,36,2,2.5,CONTROL STAKE,...,0.285714,0.222222,0,0,0,0,1,0,0,0
2,5,20190504,1,3,138700,0.0,5,3,2.5,CURATE,...,0.105263,0.000000,1,0,0,1,0,0,0,1
3,6,20190504,1,4,298244,0.0,17,4,6.0,LOOSE ON THE TOWN,...,0.300000,0.000000,0,0,0,0,0,1,0,0
4,3,20190504,1,5,183450,0.0,11,5,10.0,KOWBOY KARMA,...,0.333333,0.000000,0,0,0,0,0,0,0,0


# Standardize Data

In [202]:
def standardize_series(s):
    return (s - s.mean()) / s.std()

# Standardize all columns except explicit columns
no_std = ['race_num', 'date', 'horse_name', 'odds', 'top3', 'entry']
master_df['odds'] = master_df['ml_odds']
std_cols = [col for col in master_df.columns if col not in no_std]

for col in std_cols:
    master_df[col] = standardize_series(master_df[col]).fillna(0)

# Load Models from Disk

In [203]:
itm_file = './models/itm_rfc_abc_06215tp.joblib'
win_file = './models/win_rfc_abc_0632tp.joblib'

itm_model = load(itm_file)
win_model = load(win_file)

# First predict if the horse will be in top 3
for idx, row in master_df.iterrows():
    print('Entry: {}/{}'.format(idx+1, master_df.shape[0]), end='\r', flush=True)
    master_df.loc[idx, 'top3'] = itm_model.predict(row[std_cols].values.reshape(1,-1))
    
    # If in top3, predict if winner
    if master_df.loc[idx, 'top3'] == 1:
        master_df.loc[idx, 'win'] = win_model.predict(row[std_cols].values.reshape(1, -1))
    else:
        master_df.loc[idx, 'win'] = 0
        
    
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,recent_race,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist,odds,top3,win
0,6.093549,20190504,1,-1.353222,0.725656,0.0,3.637038,1,-0.040730,RECOUNT,...,-0.378119,0.0,1.104229,-0.623956,-0.378119,-0.175155,2.221387,12.0,0.0,0.0
1,6.093549,20190504,1,-1.131230,0.403066,0.0,3.773560,2,-1.066420,CONTROL STAKE,...,-0.378119,0.0,-0.900187,1.593080,-0.378119,-0.175155,-0.447474,2.5,1.0,1.0
2,0.966986,20190504,1,-0.909238,-0.368500,0.0,-0.458615,3,-1.066420,CURATE,...,-0.378119,0.0,1.104229,-0.623956,-0.378119,-0.175155,2.221387,2.5,1.0,1.0
3,1.287396,20190504,1,-0.687246,-0.010783,0.0,1.179646,4,-0.688534,LOOSE ON THE TOWN,...,-0.378119,0.0,-0.900187,-0.623956,2.628831,-0.175155,-0.447474,6.0,1.0,1.0
4,0.326166,20190504,1,-0.465253,-0.268165,0.0,0.360516,5,-0.256665,KOWBOY KARMA,...,-0.378119,0.0,-0.900187,-0.623956,-0.378119,-0.175155,-0.447474,10.0,1.0,0.0


# Calc 'Expected Value' of Bets

In [204]:
# EV mappings -- ev_(itm_out)_(win_out)
ev_dict = {
    'ev_1_1': 0.392,
    'ev_0_1': 0.248,
    'ev_1_0': 0.228,
    'ev_0_0': 0.153
}

for idx, row in master_df.iterrows():
    itm_val = int(row.loc['top3'])
    win_val = int(row.loc['win'])
    
    ev_val = ev_dict['ev_{}_{}'.format(itm_val, win_val)]
    master_df.loc[idx, 'EV'] = ev_val * row.loc['odds']
    
master_df.head()

,starts_at_dist,date,race_num,post_pos,lt_earnings,app_weight_alw,lt_starts,entry,ml_odds,horse_name,...,was_claimed,was_favorite,improved_stretch_pos,had_bullet,won_by_margin,won_at_similar_dist,odds,top3,win,EV
0,6.093549,20190504,1,-1.353222,0.725656,0.0,3.637038,1,-0.040730,RECOUNT,...,0.0,1.104229,-0.623956,-0.378119,-0.175155,2.221387,12.0,0.0,0.0,1.836
1,6.093549,20190504,1,-1.131230,0.403066,0.0,3.773560,2,-1.066420,CONTROL STAKE,...,0.0,-0.900187,1.593080,-0.378119,-0.175155,-0.447474,2.5,1.0,1.0,0.980
2,0.966986,20190504,1,-0.909238,-0.368500,0.0,-0.458615,3,-1.066420,CURATE,...,0.0,1.104229,-0.623956,-0.378119,-0.175155,2.221387,2.5,1.0,1.0,0.980
3,1.287396,20190504,1,-0.687246,-0.010783,0.0,1.179646,4,-0.688534,LOOSE ON THE TOWN,...,0.0,-0.900187,-0.623956,2.628831,-0.175155,-0.447474,6.0,1.0,1.0,2.352
4,0.326166,20190504,1,-0.465253,-0.268165,0.0,0.360516,5,-0.256665,KOWBOY KARMA,...,0.0,-0.900187,-0.623956,-0.378119,-0.175155,-0.447474,10.0,1.0,0.0,2.280


# Print Predictions

In [205]:
# Create a separate table for each race
cols_to_keep = ['horse_name', 'entry', 'top3', 'win', 'odds', 'EV']
out = ''
for num in range(1, master_df.race_num.max()+1):
    race_df = master_df.loc[master_df['race_num'] == num][cols_to_keep]
    out += 'RACE: {}'.format(num)
    out += '\n'
    out += tabulate(race_df, headers='keys', tablefmt='psql', showindex=False)
    out += '\n\n'
    #print('RACE: {}'.format(num))
    #print(tabulate(race_df, headers='keys', tablefmt='psql', showindex=False))
    #print('\n')
    
output_file = open('./prediction.txt', 'w+')
output_file.write(out)
output_file.close()